<a href="https://colab.research.google.com/github/wyrzykow/BHTOM-utils/blob/main/BHTOM_for_Staszek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BHTOM newletter for CDK-500 Krakow telescope

In [1]:
!pip install astroplan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for astroplan: filename=astroplan-0.10-py3-none-any.whl size=83832 sha256=02de5e74e3102cfc20432d20d95ebea967972a5ed596cfb155a627d5db313f22
  Stored in directory: /root/.cache/pip/wheels/f1/1d/bd/ce95e8776b8ebc0f8a432c8a18d9268ede4d9d5a5b24c22a6c
Successfully built astroplan


In [2]:
from astropy.coordinates import get_body, SkyCoord
from astropy import units
from astropy.time import Time
from astroplan import Observer, FixedTarget, time_grid_from_range
import numpy as np
from datetime import datetime, timedelta, date


In [3]:
#SETUP
send_email=False

airmass_limit = 2.5
length=2 #in days
interval=60 #in minutes between samplings
start_time = datetime.now()
site_details = {'sitecode': 'Krakow', 'latitude': 50.05, 'longitude': 19.82, 'elevation': 300}
observer = Observer(longitude=site_details.get('longitude')*units.deg,
                    latitude=site_details.get('latitude')*units.deg,
                    elevation=site_details.get('elevation')*units.m)


In [4]:
#BHTOM token
from google.colab import userdata
auth_token=userdata.get('bhtomtoken')

In [7]:
def get_astroplan_sun_and_time(start_time, end_time, interval):
    """
    Uses astroplan's time_grid_from_range to generate
    an astropy Time object covering the time range.

    Uses astropy's get_sun to generate sun positions over
    that time range.

    If time range is small and interval is coarse, approximates
    the sun at a fixed position from the middle of the
    time range to speed up calculations.
    Since the sun moves ~4 minutes a day, this approximation
    happens when the number of days covered by the time range
    * 4 is less than the interval (in minutes) / 2.

    :param start_time: start of the window for which to calculate the airmass
    :type start_time: datetime

    :param end_time: end of the window for which to calculate the airmass
    :type end_time: datetime

    :param interval: time interval, in minutes, at which to calculate airmass within the given window
    :type interval: int

    :returns: ra/dec positions of the sun over the time range,
        time range between start_time and end_time at interval
    :rtype: astropy SkyCoord, astropy Time
    """

    start = Time(start_time)
    end = Time(end_time)

    time_range = time_grid_from_range(time_range=[start, end], time_resolution=interval*units.minute)

    number_of_days = end.mjd - start.mjd
    if number_of_days*4 < float(interval)/2:
        # Hack to speed up calculation by factor of ~3
        sun_coords = get_body("sun",time_range[int(len(time_range)/2)])
        sun = FixedTarget(name='sun', coord=SkyCoord(sun_coords.ra, sun_coords.dec, unit='deg'))
    else:
        sun = get_body("sun", time_range)

    return sun, time_range


In [10]:

#for a given target and observatory, returns true if the object is visible in the period provided
def check_visibility(name, ra, dec, observer, start_time, length=2, interval=60, airmass_limit=2.5):
  body = FixedTarget(name=name, coord=SkyCoord(ra, dec, unit='deg'))

  end_time = start_time + timedelta(days=length)

  visibility = {}
  sun, time_range = get_astroplan_sun_and_time(start_time, end_time, interval)
  sun_alt = observer.altaz(time_range, sun).alt
  obj_airmass = observer.altaz(time_range, body).secz

  bad_indices = np.argwhere(
      (obj_airmass >= airmass_limit) |
      (obj_airmass <= 1) |
      (sun_alt > -12*units.deg)  # between astronomical twilights, i.e. sun is up
  )

  obj_airmass = [None if i in bad_indices else float(airmass) for i, airmass in enumerate(obj_airmass)]

  # Check if all elements in obj_airmass are None
  if all(i is None for i in obj_airmass):
#      print("The list obj_airmass is full of None values.")
      return False
  else:
#      print("The list obj_airmass contains non-None values.")
      return True


In [13]:
name="Gaia24bfe"
ra,dec=311.33472,-17.57744

check_visibility(name, ra, dec, observer, start_time)

False

In [ ]:
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import pytz

from IPython.display import display, HTML


In [ ]:
 #asking for ALL targets, as for now, API does not allow for filtering on importance
request_body = {
         "decMin": -30

    }

# Define headers
headers = {
    'accept': 'application/json',
    'Authorization': f'Token {auth_token}',  ## you can hard-code your token here as well
    'Content-Type': 'application/json',
    'X-CSRFToken': 'uUz2fRnXhPuvD9YuuiDW9cD1LsajeaQnE4hwtEAfR00SgV9bD5HCe5i8n4m4KcOr'
}
api_url =  "https://bh-tom2.astrolabs.pl/targets/getTargetList/"

# Send the POST request
response = requests.post(api_url, json=request_body, headers=headers)

# Assuming response.text is your JSON data
data_json = json.loads(response.text)

data = json.loads(data_json)

#fields = data[0]['fields']

# Extracting field values into a list of dictionaries
data_list = [item['fields'] for item in data]

# Creating DataFrame from the list
df = pd.DataFrame(data_list)

#print(df)


In [ ]:
df.columns

Index(['name', 'type', 'created', 'modified', 'ra', 'dec', 'epoch', 'parallax',
       'pm_ra', 'pm_dec', 'galactic_lng', 'galactic_lat', 'distance',
       'distance_err', 'scheme', 'epoch_of_elements', 'mean_anomaly',
       'arg_of_perihelion', 'eccentricity', 'lng_asc_node', 'inclination',
       'mean_daily_motion', 'semimajor_axis', 'epoch_of_perihelion',
       'ephemeris_period', 'ephemeris_period_err', 'ephemeris_epoch',
       'ephemeris_epoch_err', 'perihdist', 'classification', 'discovery_date',
       'mjd_last', 'mag_last', 'importance', 'cadence', 'priority',
       'sun_separation', 'creation_date', 'constellation', 'dont_update_me',
       'phot_class', 'photometry_plot', 'photometry_plot_obs',
       'photometry_icon_plot', 'spectroscopy_plot', 'data_plot', 'filter_last',
       'cadence_priority', 'description'],
      dtype='object')

In [ ]:
#filtering on importance and sundistance
df0 = df[(df.importance > 1)].copy().reset_index(drop=True)

df1 = df[(df.importance > 4) & (df.sun_separation > 50) & (df.mag_last<18)].copy().reset_index(drop=True)

print(len(df0))
print(len(df1))



154
44


In [ ]:

# Convert 'created' column to datetime
df0['created'] = pd.to_datetime(df0['created'])
df1['created'] = pd.to_datetime(df1['created'])

# Get the date one week ago in UTC
one_week_ago = datetime.now(pytz.UTC) - timedelta(weeks=1)
current_date = datetime.now(pytz.UTC)
formatted_date = current_date.strftime("%d %B, %Y")

# Filter rows where 'created' is later than one week ago
df_lastweek = df0[df0['created'] > one_week_ago]
df_lastweek_sorted = df_lastweek.sort_values('mag_last')

df_old = df1[df1['created']<one_week_ago] #DF1 is also filtered by 18 mag
df_old_north = df_old[df_old['dec']>0]
df_old_south = df_old[df_old['dec']<=0]

df_old_N_sorted = df_old_north.sort_values('mag_last')
df_old_S_sorted = df_old_south.sort_values('mag_last')

df_lastweek_sorted

,name,type,created,modified,ra,dec,epoch,parallax,pm_ra,pm_dec,...,dont_update_me,phot_class,photometry_plot,photometry_plot_obs,photometry_icon_plot,spectroscopy_plot,data_plot,filter_last,cadence_priority,description
153,ZTF23aaqazhf,SIDEREAL,2024-05-30 18:09:48.023000+00:00,2024-05-31T08:43:07.409Z,285.385319,22.614975,2000.0,None,None,None,...,None,Red Giant 94.8%,/plots/photometry/ZTF23aaqazhf_plot.json,/plots/photometry/obs_ZTF23aaqazhf_plot.json,/plots/photometryIcon/ZTF23aaqazhf_plot.json,,None,~G,10.4,"potential microlensing event from FINK, KATS24..."


In [ ]:
# Selecting the required columns
df_lastweek_selected = df_lastweek_sorted[['name', 'ra', 'dec', 'mag_last', 'sun_separation', 'classification', 'description']]
df_old_N_selected = df_old_N_sorted[['name', 'ra', 'dec', 'mag_last', 'sun_separation', 'classification', 'description']]
df_old_S_selected = df_old_S_sorted[['name', 'ra', 'dec', 'mag_last', 'sun_separation', 'classification', 'description']]

# Define a function to format the strings
def make_hyperlink(name):
    return '<a href="https://bh-tom2.astrolabs.pl/targets/{0}/">{0}</a>'.format(name)

df_lastweek_selected = df_lastweek_selected.copy()
df_lastweek_selected.loc[:, 'name'] = df_lastweek_selected['name'].apply(make_hyperlink)

df_old_N_selected = df_old_N_selected.copy()
df_old_N_selected.loc[:, 'name'] = df_old_N_selected['name'].apply(make_hyperlink)

df_old_S_selected = df_old_S_selected.copy()
df_old_S_selected.loc[:, 'name'] = df_old_S_selected['name'].apply(make_hyperlink)

# Convert the DataFrame to HTML and escape=False to render HTML tags
html_table = df_lastweek_selected.to_html(index=False, escape=False)
# Remove border by replacing <table> with <table border="0">
html_table = html_table.replace('<table', '<table border="0"')

old_N_table = df_old_N_selected.to_html(index=False, escape=False)
old_N_table = old_N_table.replace('<table', '<table border="0"')
old_S_table = df_old_S_selected.to_html(index=False, escape=False)
old_S_table = old_S_table.replace('<table', '<table border="0"')

link_N = "https://bh-tom2.astrolabs.pl/targets/?type=&name=&classification=&description=&cone_search=&targetlist__name=&target_cone_search=&ra_min=&ra_max=&dec_min=0&dec_max=&gall_min=&gall_max=&galb_min=&galb_max=&importance_min=4&importance_max=&priority_min=&priority_max=&sun_min=70&sun_max=&mag_min=&mag_max=18&order=mag_last"
link_S = "https://bh-tom2.astrolabs.pl/targets/?type=&name=&classification=&description=&cone_search=&targetlist__name=&target_cone_search=&ra_min=&ra_max=&dec_min=&dec_max=0&gall_min=&gall_max=&galb_min=&galb_max=&importance_min=4&importance_max=&priority_min=&priority_max=&sun_min=70&sun_max=&mag_min=&mag_max=18&order=mag_last"



In [ ]:
#asking for dataproducts from last week
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Get the current date and time
now = datetime.now()

# Calculate the date a week ago
week_ago = now - relativedelta(weeks=1)

# Format the date as a string in the required format
week_ago_str = week_ago.strftime("%Y-%m-%dT%H:%M:%S.%f%z")
beginning_2024 = "2024-01-01T00:00:00.00+00:00"
date = "2024-05-01T00:00:00.00+00:00"
#

request_body = {
    "created_start": week_ago_str,
}

# request_body = {
#     "created_start":"2023-01-01T00:06:23.653199+01:00"
#     }

# Define headers
headers = {
    'accept': 'application/json',
    'Authorization': f'Token {auth_token}',  ## you can hard-code your token here as well
    'Content-Type': 'application/json',
    'X-CSRFToken': 'uUz2fRnXhPuvD9YuuiDW9cD1LsajeaQnE4hwtEAfR00SgV9bD5HCe5i8n4m4KcOr'
}
api_url =  "https://bh-tom2.astrolabs.pl/common/api/data/"

# Send the POST request
response = requests.post(api_url, json=request_body, headers=headers)

# Assuming response.text is your JSON data
data_json = json.loads(response.text)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_json)


In [ ]:
df = pd.DataFrame(data_json)
df.drop(df[df['dryRun'] == True].index, inplace=True)

df.columns

Index(['id', 'user_name', 'target_name', 'target', 'user', 'camera',
       'observatory_name', 'observatory', 'product_id', 'data', 'status',
       'photometry_data', 'fits_data', 'extra_data', 'created', 'modified',
       'data_product_type', 'featured', 'thumbnail', 'dryRun', 'comment',
       'observation_record', 'group'],
      dtype='object')

In [ ]:
# #grabbing observatory list:
# # Define headers
# request_body = {
#          "decMin": -1000

#     }

# headers = {
#     'accept': 'application/json',
#     'Authorization': f'Token {auth_token}',  ## you can hard-code your token here as well
#     'Content-Type': 'application/json',
#     'X-CSRFToken': 'uUz2fRnXhPuvD9YuuiDW9cD1LsajeaQnE4hwtEAfR00SgV9bD5HCe5i8n4m4KcOr'
# }
# api_url =  "https://bh-tom2.astrolabs.pl/observatory/getObservatoryList/"

# # Send the POST request
# response = requests.post(api_url, json=request_body, headers=headers)
# #response

# # # Assuming response.text is your JSON data
# data_json = json.loads(response.text)

# data = json.loads(data_json)

# #fields = data[0]['fields']

# # Extracting field values into a list of dictionaries
# data_list = [item['fields'] for item in data]

# # Creating DataFrame from the list
# df_obs = pd.DataFrame(data_list)

In [ ]:
# df_obs.columns

In [ ]:
total_fits = len(df)
total_fits

5301

In [ ]:
# #adding prefix column to my main df with observations:

# #note that df already contains name and prefix, but are all Nans - why?
# # Merge df with df_obs on 'observatory' and 'name'
# merged_df = pd.merge(df, df_obs[['name', 'prefix']], left_on='observatory', right_on='name', how='left')

# merged_df.drop('name', axis=1, inplace=True)
# merged_df = merged_df.rename(columns={'prefix': 'observatory_prefix'})

# # If you want to replace the original df with the merged one
# df = merged_df


In [ ]:
# Create a new column 'observatory_user' that combines 'observatory' and 'user'
df['observatory_user'] = df['user_name'] + " ("+df['camera'] + ")"

# Count the occurrences of each unique 'observatory_user', and get the top 5
top_5 = df['user_name'].value_counts().head(5)
top_5_df = top_5.reset_index()
top_5_df.columns = ['name', 'count']
top_5_html = top_5_df.to_html(index=False)

top_5_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>name</th>\n      <th>count</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Rachel Street</td>\n      <td>1825</td>\n    </tr>\n    <tr>\n      <td>Franz-Josef Hambsch</td>\n      <td>1370</td>\n    </tr>\n    <tr>\n      <td>Supachai Awiphan</td>\n      <td>945</td>\n    </tr>\n    <tr>\n      <td>Katarzyna Kruszynska</td>\n      <td>271</td>\n    </tr>\n    <tr>\n      <td>Sjoerd Dufoer</td>\n      <td>239</td>\n    </tr>\n  </tbody>\n</table>'

In [ ]:
df['created'] = pd.to_datetime(df['created'])
one_week_ago = datetime.now(pytz.UTC) - timedelta(weeks=1)
df_last_week = df[df['created'] > one_week_ago]
#TEMP: ALL
#df_last_week = df
last_week_counts = df_last_week['observatory_user'].value_counts()


In [ ]:
# Group by 'target_name' and apply join on unique 'user_name'
df_grouped = df_last_week.groupby('target_name')['user_name'].apply(lambda x: ', '.join(set(x))).reset_index()

# Rename the columns as per your requirement
df_grouped.columns = ['target_name', 'user_names']

# Convert DataFrame to HTML, remove index, justify all columns to the right
html_grouped = df_grouped.to_html(index=False)

# Add style to justify text to the right for all columns
html_grouped = html_grouped.replace('<table', '<table border="0" style="text-align: right;"')

# Display HTML string in Google Colab
display(HTML(html_grouped))


target_name,user_names
8C0716_714,Justas Zdanavicius
ASASSN-14cc,Magdalena Szkudlarek
Gaia14aaf,Chris OHare
Gaia19bpg,Adam Popowicz
Gaia21asp,Adam Popowicz
Gaia21ccu,Franz-Josef Hambsch
Gaia22alz,Staszek Zola
Gaia22bpl,Supachai Awiphan
Gaia23ats,Supachai Awiphan
Gaia23bay,Supachai Awiphan


In [ ]:
# Group by 'observatory' and apply join on unique 'target_name'
df_grouped_obs = df_last_week.groupby('camera')['target_name'].apply(lambda x: ', '.join(set(x))).reset_index()

# Apply the make_hyperlink function to each target_name
#df_grouped_obs['target_name'] = df_grouped_obs['target_name'].str.split(', ').apply(lambda names: ', '.join(make_hyperlink(name) for name in names))

# Rename the columns as per your requirement
df_grouped_obs.columns = ['camera', 'target_names']

# Convert DataFrame to HTML, remove index, justify all columns to the right
html_grouped_obs = df_grouped_obs.to_html(index=False)

# Add style to justify text to the right for all columns
html_grouped_obs = html_grouped_obs.replace('<table', '<table border="0" style="text-align: right;"')

# Display HTML string in Google Colab
display(HTML(html_grouped_obs))
#df_grouped_obs

camera,target_names
Adonis_G2-1600,"Gaia24bfe, Gaia24ayd, Gaia24azc, Gaia24azi"
CAHA1.23_ASI461MM,"Gaia24azc, TCrB, Gaia24azi, SN 2024igg, NGC5683-Seyfert"
EEye_QHY268M,"Gaia24azi, Gaia24ayd, Gaia24bfe, Gaia23cpd"
ElSauce_QHY600MPro,Gaia24azi
Flarestar-MPC171_G2-1600,"TCrB, TabbysStar, Gaia24azi"
LCOGT-CTIO-1m_4K,"Gaia24ata, ZTF23aaqazhf, Gaia23dpn, Gaia24azi, Gaia24amo, Gaia23dpi, Gaia24asr"
LCOGT-HO-2m_Spectral,"Gaia24acn, Gaia24bhf, Gaia24azi"
LCOGT-MCD-1m_4K,"ZTF23aaqazhf, Gaia24acn, Gaia24azi"
LCOGT-SAAO-1m_4K,"Gaia24amk, Gaia23dpi, ZTF24aaiafkl, Gaia24asr, Gaia23dpd"
LCOGT-SS-1m_4K,"Gaia23dpn, Gaia24azi, Gaia24amk, Gaia23dpi, Gaia23cxu"


In [ ]:

# Convert Series to DataFrame and rename columns
df = last_week_counts.to_frame().reset_index()
df.columns = ['observatory-user', 'count']

# Convert DataFrame to HTML, remove index, justify all columns to the right
html_last_week_count = df.to_html(index=False)

# Add style to justify text to the right for all columns
html_last_week_count = html_last_week_count.replace('<table', '<table border="0" style="text-align: right;"')

# Display HTML string in Google Colab
display(HTML(html_last_week_count))


observatory-user,count
Franz-Josef Hambsch (ROAD_QHY600M),1370
Rachel Street (LCOGT-CTIO-1m_4K),762
Supachai Awiphan (TRT-SRO-0.7_Andor-934),472
Rachel Street (LCOGT-SS-1m_4K),372
Rachel Street (LCOGT-MCD-1m_4K),355
Rachel Street (LCOGT-Teide-1m_4K),332
Supachai Awiphan (TRT-SBO-0.7_Andor-934),330
Sjoerd Dufoer (EEye_QHY268M),239
Katarzyna Kruszynska (LCOGT-SS-2m_Spectral),144
Supachai Awiphan (TRT-CTO-0.7_Andor-934),143


In [ ]:
# prompt: from df_last_week list count of unique entries in target column

unique_targets = df_last_week['target_name'].value_counts()

unique_targets

target_name
Gaia24azi            3049
Gaia24acn             426
Gaia23bay             324
Gaia21ccu             179
Gaia24bfr             139
Gaia23dpn             130
TCrB                  110
Gaia23ats             101
Gaia23bor              93
Gaia24asr              90
Gaia23dgt              85
Gaia23ckh              75
Gaia21asp              63
ZTF23aaqazhf           51
Gaia24bau              40
Gaia22bpl              39
Gaia23cpd              38
Gaia24bfe              35
Gaia24ayd              34
Gaia19bpg              32
Gaia24bhu              18
TabbysStar             15
Gaia24bhf              13
Gaia24azc              12
Gaia24bct              12
Gaia23dpi               9
Gaia24amk               8
Gaia22alz               8
Gaia24beh               8
Gaia24aup               8
Gaia24avs               7
Gaia24amo               7
ASASSN-14cc             6
Gaia24bic               6
8C0716_714              6
ZTF24aaiafkl            4
SN 2024igg              4
Gaia24biz               3


In [ ]:
df_last_week

,id,user_name,target_name,target,user,camera,observatory_name,observatory,product_id,data,...,created,modified,data_product_type,featured,thumbnail,dryRun,comment,observation_record,group,observatory_user
0,169502,Steve Fossey,Gaia24azi,3012,steve@fossey.org.uk,UCLO-C14EAST_PL9000,UCLO C14 EAST 36-cm,47.0,None,/data/fits/Gaia24azi/45_2024_05_27T23_42_59_Ga...,...,2024-05-28 08:30:54.635518+02:00,2024-05-28T08:54:43.766769+02:00,fits_file,False,None,False,None,None,[],Steve Fossey (UCLO-C14EAST_PL9000)
1,169503,Steve Fossey,Gaia24azi,3012,steve@fossey.org.uk,UCLO-C14EAST_PL9000,UCLO C14 EAST 36-cm,47.0,None,/data/fits/Gaia24azi/45_2024_05_27T23_46_42_Ga...,...,2024-05-28 08:30:54.865673+02:00,2024-05-28T08:54:45.055965+02:00,fits_file,False,None,False,None,None,[],Steve Fossey (UCLO-C14EAST_PL9000)
2,169504,Steve Fossey,Gaia24azi,3012,steve@fossey.org.uk,UCLO-C14EAST_PL9000,UCLO C14 EAST 36-cm,47.0,None,/data/fits/Gaia24azi/45_2024_05_27T23_50_23_Ga...,...,2024-05-28 08:30:55.031039+02:00,2024-05-28T09:14:44.915648+02:00,fits_file,False,None,False,None,None,[],Steve Fossey (UCLO-C14EAST_PL9000)
3,169505,Steve Fossey,Gaia24azi,3012,steve@fossey.org.uk,UCLO-C14EAST_PL9000,UCLO C14 EAST 36-cm,47.0,None,/data/fits/Gaia24azi/45_2024_05_28T00_07_35_Ga...,...,2024-05-28 08:32:53.298382+02:00,2024-05-28T08:50:43.327850+02:00,fits_file,False,None,False,None,None,[],Steve Fossey (UCLO-C14EAST_PL9000)
4,169506,Steve Fossey,Gaia24azi,3012,steve@fossey.org.uk,UCLO-C14EAST_PL9000,UCLO C14 EAST 36-cm,47.0,None,/data/fits/Gaia24azi/45_2024_05_28T00_11_22_Ga...,...,2024-05-28 08:32:53.482952+02:00,2024-05-28T09:14:25.705550+02:00,fits_file,False,None,False,None,None,[],Steve Fossey (UCLO-C14EAST_PL9000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,175302,Uliana Pylypenko,TCrB,2693,uliana,CAHA1.23_ASI461MM,CAHA 1.23-m Telescope,160.0,None,/data/fits/TCrB/114_TCrB_22_05_2024T20_06_15_0...,...,2024-06-03 20:39:17.115779+02:00,2024-06-03T20:40:01.913731+02:00,fits_file,False,None,False,None,None,[],Uliana Pylypenko (CAHA1.23_ASI461MM)
5469,175303,Uliana Pylypenko,TCrB,2693,uliana,CAHA1.23_ASI461MM,CAHA 1.23-m Telescope,160.0,None,/data/fits/TCrB/114_TCrB_22_05_2024T19_59_22_0...,...,2024-06-03 20:39:34.654388+02:00,2024-06-03T20:40:18.493293+02:00,fits_file,False,None,False,None,None,[],Uliana Pylypenko (CAHA1.23_ASI461MM)
5470,175304,Uliana Pylypenko,NGC5683-Seyfert,2865,uliana,CAHA1.23_ASI461MM,CAHA 1.23-m Telescope,160.0,None,/data/fits/NGC5683_Seyfert/114_NGC5683_Seyfert...,...,2024-06-03 21:36:52.312266+02:00,2024-06-03T21:37:38.829934+02:00,fits_file,False,None,False,None,None,[],Uliana Pylypenko (CAHA1.23_ASI461MM)
5471,175305,Uliana Pylypenko,NGC5683-Seyfert,2865,uliana,CAHA1.23_ASI461MM,CAHA 1.23-m Telescope,160.0,None,/data/fits/NGC5683_Seyfert/114_NGC5683_Seyfert...,...,2024-06-03 21:39:56.059625+02:00,2024-06-03T21:40:43.582053+02:00,fits_file,False,None,False,None,None,[],Uliana Pylypenko (CAHA1.23_ASI461MM)


In [ ]:
# Get counts of each status
status_counts = df_last_week['status'].value_counts()

# Print counts
print(status_counts)

# Count for success statuses
success_count = status_counts.get('S', 0)
print(f'Success count: {success_count}')

# Count for error statuses
error_count = status_counts.get('E', 0)
print(f'Error count: {error_count}')

# Count for other statuses
other_count = status_counts.sum() - success_count - error_count
print(f'Other statuses count: {other_count}')

status
S    4797
E     479
P      21
R       4
Name: count, dtype: int64
Success count: 4797
Error count: 479
Other statuses count: 25


#formating the email

In [ ]:
# Creating the email message
email_message = f"""
Hello,
<p>
Greetings from the BHTOM Automated Newsletter!
</p>
<p>

</p>
<p>
As of {current_date}, these are the new targets added in the last week with importance greater than 1, sorted by magnitude:
</p>
<p>
{html_table}
</p>
<p>
In addition, here are some older targets that are currently visible and requested for observations. These targets have an importance greater than 4, a sun separation greater than 70, and a magnitude less than 18. They are also sorted by magnitude.
</p>
<hr>
<p>
<b><a href="{link_N}">North (dec>0):</a></b>
<br>
{old_N_table}
</p>
<hr>
<p>
<b><a href="{link_S}">South (dec<0):</a></b>
<br>
{old_S_table}
</p>
<hr>
<p>
<b>Last week's total of images processed: {total_fits} ({success_count} with success).</b>
<hr>
<p>
<b>Last week's targets observed:</b>
<br>
{html_grouped_obs}
</p>
<p>
<b>Last week's fits uploads score</b> (sorted by count)
<br>
{html_last_week_count}
</p>
<p>
<p>
<hr>
<b>Top 5 data uploaders!</b>
{top_5_html}
<p>
<p>
<hr>
Clear skies!
<br>
Your BHTOM Team
<br>
<a href="http://bhtom.space">bhtom.space</a>
</p>
"""#.format(current_date, html_table, old_N_table, old_S_table, link_N, link_S, html_grouped_obs, html_last_week_count)


In [ ]:
email_message

'\nHello,\n<p>\nGreetings from the BHTOM Automated Newsletter!\n</p>\n<p>\n\n</p>\n<p>\nAs of 2024-06-04 05:26:33.080591+00:00, these are the new targets added in the last week with importance greater than 1, sorted by magnitude:\n</p>\n<p>\n<table border="0" border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>name</th>\n      <th>ra</th>\n      <th>dec</th>\n      <th>mag_last</th>\n      <th>sun_separation</th>\n      <th>classification</th>\n      <th>description</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td><a href="https://bh-tom2.astrolabs.pl/targets/ZTF23aaqazhf/">ZTF23aaqazhf</a></td>\n      <td>285.385319</td>\n      <td>22.614975</td>\n      <td>15.9</td>\n      <td>125.0</td>\n      <td>Unknown</td>\n      <td>potential microlensing event from FINK, KATS24K002</td>\n    </tr>\n  </tbody>\n</table>\n</p>\n<p>\nIn addition, here are some older targets that are currently visible and requested for observations. These targets hav

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from getpass import getpass

if send_email:
  # Set up the SMTP server
  # Set up the SMTP server
  s = smtplib.SMTP(host='smtp.gmail.com', port=587)
  s.starttls()

  # Login to your Gmail account
  password = 'ggzr clob tbcm ehtc'
  s.login('wyrzykow@gmail.com', password)

  # Create a message
  msg = MIMEMultipart()  # create a message

  # Setup the parameters of the message
  msg['From']="wyrzykow@gmail.com"
  msg['To']="bhtomtargets@googlegroups.com"
  msg['Subject']=f"BHTOM Targets for {formatted_date}"

  msg.attach(MIMEText(email_message, 'html'))

  # Send the message via the server set up earlier.
  s.send_message(msg)

  # Terminate the SMTP session and close the connection
  s.quit()


In [ ]:
### Separate code for BHTOM webpage:
# recently observed events - target names
# one-year statistics: winners per users


In [ ]:
display(HTML(email_message))

name,ra,dec,mag_last,sun_separation,classification,description
ZTF23aaqazhf,285.385319,22.614975,15.9,125.0,Unknown,"potential microlensing event from FINK, KATS24K002"
name,ra,dec,mag_last,sun_separation,classification,description
TCrB,239.875676,25.920170,10.9,130.0,Nova,recurent nova predicted to explode 2024/2025
4U2206+54,331.984320,54.518447,11.3,78.0,XRB,Be X-ray binary
Gaia24ayd,300.825090,30.651260,12.9,109.0,Unknown,bright candidate for microlensing event
SAXJ2103.5+4545,315.898792,45.751547,13.7,91.0,XRB,Be X-ray binary
IGRJ21343+4738,323.584883,47.633391,14.0,85.0,XRB,Be X-ray binary
KS1947+300,297.397850,30.208827,14.1,112.0,XRB,Be X-ray binary
GROJ2058+42,314.698071,41.776995,14.3,94.0,XRB,Be X-ray binary
SN 2024igg,227.378793,54.505622,14.6,100.0,SN,Astro-COLIBRI target


In [ ]:
## from last week's targets - identify these with importance = 0 and inform observed to stop observing
# or: list targets where importance changed from >0 to 0 in the last week - how to trace it ??
# history of importance?